# Выявление негативных комментарий

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других.


**Цель проекта:**

Обучить модель классифицировать комментарии на позитивные и негативные на основе набора данных с разметкой о токсичности правок. Метрика качества *F1* должна быть не меньше 0.75. 

**Задачи:**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from pymystem3 import Mystem
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from sklearn.feature_extraction.text import TfidfVectorizer 
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arnam\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arnam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


In [2]:
df = pd.read_csv('toxic_comments.csv')
display(df.head())
df.info(memory_usage='deep')

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 80.1 MB


In [3]:
df['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

In [4]:
print(f"Процент объектов класса 1 к общему объёму датасета: {(sum(df['toxic']) / len(df) * 100):.2f}%")

Процент объектов класса 1 к общему объёму датасета: 10.17%


Проведем лемматизацию текстов

In [5]:
def lemmatize(text):
    wt = nltk.word_tokenize(text)
    lem_join = " ".join([WordNetLemmatizer().lemmatize(w) for w in wt])
    return lem_join
    
df['lemm_text'] = df['text'].apply(lemmatize)
df = df.drop(['text'], axis=1)

In [6]:
df['lemm_text']

0         Explanation Why the edits made under my userna...
1         D'aww ! He match this background colour I 'm s...
2         Hey man , I 'm really not trying to edit war ....
3         `` More I ca n't make any real suggestion on i...
4         You , sir , are my hero . Any chance you remem...
                                ...                        
159566    `` : : : : : And for the second time of asking...
159567    You should be ashamed of yourself That is a ho...
159568    Spitzer Umm , there no actual article for pros...
159569    And it look like it wa actually you who put on...
159570    `` And ... I really do n't think you understan...
Name: lemm_text, Length: 159571, dtype: object

Выделим обучающую и тестовую выборки

In [7]:
features = df.drop(['toxic'], axis=1)
target = df['toxic']

features_train, features_test, target_train, target_test = train_test_split(features, target,
                                                                              test_size=0.2, random_state=42)

Создадим мешок слов с учётом стоп-слов

In [8]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords) 

features_train = count_tf_idf.fit_transform(features_train['lemm_text'].values.astype('U'))
features_test = count_tf_idf.transform(features_test['lemm_text'].values.astype('U'))

features_train.shape, features_test.shape

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arnam\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


((127656, 161957), (31915, 161957))

**Вывод:**

В данном пункте для подготовки данных данные обучению были произведены лемматизация текстов, разделение данных на обучающую и тестовую выборки, а также создан мешок слов с учетом стоп-слов.

## Обучение

Для обучения используем модели LogisticRegression, CatBoost, DecisionTreeClassifier.

**Модель LogisticRegression**

In [9]:
param_lr = [{'C':[12,13,14]}]
model_regression = LogisticRegression(random_state=42)
clf = GridSearchCV(model_regression, param_lr, scoring = 'f1', cv=5)
clf.fit(features_train, target_train)
f1_lr_cv = clf.best_score_
print(f"Наилучший показатель f1 на кросс-валидации : {f1_lr_cv:.2f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.76
Параметр регуляризации для лучшей модели: {'C': 12}


In [10]:
%%time
model_regression = LogisticRegression(C = 14, random_state=42)
model_regression.fit(features_train, target_train)
predict = model_regression.predict(features_test)
f1_lr = f1_score(target_test, predict)

print(f"Показатель f1 на тестовой выборке: {f1_lr:.2f}")

Показатель f1 на тестовой выборке: 0.78
Wall time: 6.35 s


**Модель CatBoost**

In [11]:
%%time
model_CBC = CatBoostClassifier(verbose=False, iterations=100)
model_CBC.fit(features_train, target_train)
target_predict = model_CBC.predict(features_test)
f1_CBC = f1_score(target_test, target_predict)

print(f"Показатель f1 на тестовой выборке: {f1_CBC:.2f}")

Показатель f1 на тестовой выборке: 0.74
Wall time: 2min 39s


**Модель DecisionTreeClassifier**

In [12]:
param_dtc = {'max_depth': range(4,11,3)}

model_DTC = DecisionTreeClassifier(random_state=42)
clf = GridSearchCV(model_DTC, param_dtc, scoring = 'f1', cv=5)
clf.fit(features_train, target_train)
f1_dtc_cv = clf.best_score_
print(f"Наилучший показатель f1 на кросс-валидации : {f1_dtc_cv:.2f}")
print(f"Параметр регуляризации для лучшей модели: {clf.best_params_}")

Наилучший показатель f1 на кросс-валидации : 0.58
Параметр регуляризации для лучшей модели: {'max_depth': 10}


In [13]:
%%time
model_DTC = DecisionTreeClassifier(max_depth = 7, random_state=42)
model_DTC.fit(features_train, target_train)
predict = model_DTC.predict(features_test)
f1_DTC = f1_score(target_test, predict)
print(f"Показатель f1 на тестовой выборке: {f1_DTC:.2f}")

Показатель f1 на тестовой выборке: 0.55
Wall time: 4.08 s


**Вывод** 

В данном пункте были обучены модели LogisticRegression, CatBoost, DecisionTreeClassifier. Результаты F1 данных моделей приведены в таблице ниже:

In [14]:
d = ({ 'Показатель f1 на тестовой выборке':[f1_lr, f1_CBC, f1_DTC]
      })
data = pd.DataFrame(data=d, index=['LogisticRegression','CatBoost','DecisionTreeClassifier'])
data

,Показатель f1 на тестовой выборке
LogisticRegression,0.778044
CatBoost,0.741918
DecisionTreeClassifier,0.553826


По качеству метрики f1 по времени наилучшей моделью оказалась LogisticRegression (f1 = 0.78).

## Выводы

В ходе выполнения проекта о построении модели для выявления токсичных комментариев сделано следующее:

1) подготовлены данные к обучению. Проезведена лемматизация текстов, разделение данных на обучающую и тестовую выборки, а также создан мешок слов с учетом стоп-слов.

2) обучены модели LogisticRegression, CatBoost, DecisionTreeClassifier. По качеству метрики f1 и по времени наилучшей моделью оказалась LogisticRegression (f1 = 0.78).